In [1]:
# repo root
%cd /work

/work


In [112]:
!ls 

LICENSE    adapters  examples  paper	       resources  src
README.md  docker    out       pyproject.toml  scripts	  tests


In [121]:
#!chmod +x resources/gene_id_maps/build_entrez_id_map.sh
#! ./resources/gene_id_maps/build_entrez_id_map.sh

[build_entrez_id_map] Downloading: https://ftp.ncbi.nlm.nih.gov/gene/DATA/GENE_INFO/Mammalia/Homo_sapiens.gene_info.gz
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 5033k  100 5033k    0     0  8765k      0 --:--:-- --:--:-- --:--:-- 8754k
[build_entrez_id_map] Decompressing: Homo_sapiens.gene_info.gz
[build_entrez_id_map] Writing mapping TSV...
[build_entrez_id_map] Compressing: id_map.tsv
[build_entrez_id_map] Writing checksums...
OK
  - resources/gene_id_maps/id_map.tsv.gz
  - resources/gene_id_maps/checksums.sha256

SampleCard config example:
  extra:
    gene_id_map_tsv: "resources/gene_id_maps/id_map.tsv.gz"


In [123]:
#!chmod +x resources/gene_id_maps/build_ensembl_to_symbol_from_gtf.sh
#! ./resources/gene_id_maps/build_ensembl_to_symbol_from_gtf.sh --download

[build_ensembl_id_map] Downloading: https://ftp.ensembl.org/pub/release-109/gtf/homo_sapiens/Homo_sapiens.GRCh38.109.gtf.gz
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 51.7M  100 51.7M    0     0  3839k      0  0:00:13  0:00:13 --:--:-- 4519k
OK
  - resources/gene_id_maps/ensembl_id_map.tsv.gz
  - resources/gene_id_maps/checksums.sha256

SampleCard config example:
  extra:
    gene_id_map_tsv: "resources/gene_id_maps/ensembl_id_map.tsv.gz"


In [3]:
# Fetch from UCSC Xena
!python paper/scripts/fig2_fetch_inputs.py

[fetch_inputs] OK
  /work/paper/source_data/PANCAN_TP53_v1/raw/expression.xena.gz
  /work/paper/source_data/PANCAN_TP53_v1/raw/mc3.v0.2.8.PUBLIC.xena.gz
  /work/paper/source_data/PANCAN_TP53_v1/raw/TCGA_phenotype_dense.tsv.gz


In [4]:
# Extract TP53 mutations (mc3.v0.2.8.PUBLIC.xena.gz)
# Create barcode→cancer from phenotype 
# Output derived/groups/{CANCER}.groups.tsv　for each cancer type

!python paper/scripts/fig2_make_groups.py

[make_groups] sample_type_id raw unique (head): ['07', '06', '05', '03', '02', '01', '11', '']
[make_groups] sample_type_id norm counts: {'1': 10593, '11': 1475, '6': 396, '3': 200, '': 72, '2': 56, '5': 11, '7': 1}
[make_groups] filtered to primary-like sample_type_id in {1,3}: 10793
[make_groups] phenotype mapped samples: 10793
[make_groups] cancers: ['ACC', 'BLCA', 'BRCA', 'CESC', 'CHOL', 'COAD', 'DLBC', 'ESCA', 'GBM', 'HNSC', 'KICH', 'KIRC', 'KIRP', 'LAML', 'LGG', 'LIHC', 'LUAD', 'LUSC', 'MESO', 'OV', 'PAAD', 'PCPG', 'PRAD', 'READ', 'SARC', 'SKCM', 'STAD', 'TGCT', 'THCA', 'THYM', 'UCEC', 'UCS', 'UVM']
[make_groups] MC3 TP53 rows: 3729
[make_groups] MC3 TP53 protein-altering rows (pre-FILTER): 3620
[make_groups] FILTER column not found; not applying FILTER
[make_groups] WARNING: LAML has n_mut=0, n_wt=200
[make_groups] WARNING: UVM has n_mut=0, n_wt=80
[make_groups] OK
  wrote: /work/paper/source_data/PANCAN_TP53_v1/derived/groups/PANCAN.groups.tsv
  cancers: 33
  samples (mapped): 

In [125]:
!python paper/scripts/fig2_make_sample_cards.py

[make_sample_cards] OK
  wrote: /work/paper/source_data/PANCAN_TP53_v1/sample_cards (33 cancers × 2 cards)


In [6]:
for c in ("HNSC", "LUAD", "LUSC", "BRCA", "OV",  "UCEC",  "SKCM"):
  !Rscript paper/scripts/fig2_deg_rank.R $c


[deg_rank] inputs
  cancer: HNSC 
  expr: /work/paper/source_data/PANCAN_TP53_v1/raw/expression.xena.gz 
  groups: /work/paper/source_data/PANCAN_TP53_v1/derived/groups/HNSC.groups.tsv 
[deg_rank] matched samples: 520  / expr= 11069  / groups= 528 
[deg_rank] QC: dropping zero-variance genes: 269 
[deg_rank] OK
  wrote: /work/paper/source_data/PANCAN_TP53_v1/derived/rankings/HNSC.deg_ranking.tsv 
  n_genes: 20262 
[deg_rank] inputs
  cancer: LUAD 
  expr: /work/paper/source_data/PANCAN_TP53_v1/raw/expression.xena.gz 
  groups: /work/paper/source_data/PANCAN_TP53_v1/derived/groups/LUAD.groups.tsv 
[deg_rank] matched samples: 515  / expr= 11069  / groups= 520 
[deg_rank] QC: dropping zero-variance genes: 338 
[deg_rank] OK
  wrote: /work/paper/source_data/PANCAN_TP53_v1/derived/rankings/LUAD.deg_ranking.tsv 
  n_genes: 20193 
[deg_rank] inputs
  cancer: LUSC 
  expr: /work/paper/source_data/PANCAN_TP53_v1/raw/expression.xena.gz 
  groups: /work/paper/source_data/PANCAN_TP53_v1/derived/gr

In [7]:
!Rscript paper/scripts/fig2_fgsea_to_evidence_table.R HNSC


[fgsea_to_evidence] inputs
  cancer: HNSC 
  rank: /work/paper/source_data/PANCAN_TP53_v1/derived/rankings/HNSC.deg_ranking.tsv 
  out : /work/paper/source_data/PANCAN_TP53_v1/evidence_tables/HNSC.evidence_table.tsv 
  msigdbr_version: 25.1.1 
  fgsea_version  : 1.36.2 
[fgsea_to_evidence] stats diagnostics
  stats_len: 20262 
  stats_type: double 
  stats_na: 0 
  stats_range: -12.6721379948078,10.5253182402821 
  gene_example: 4060,6817,5153,1902,19594,9055,6447,16500,1970,8301 
  gene_id_type: entrez
 [100%] Downloaded 33730584 bytes...
[fgsea_to_evidence] msigdbr entrez column: ncbi_gene 
[fgsea_to_evidence] overlap diagnostics
  overlap_summary: min 24 p25 80 median 141 p75 165 max 193 
  pathways_with_overlap>=15: 50 / 50 
[fgsea_to_evidence] fgsea diagnostics
  n_pathways: 50 
  res_rows  : 50 
[fgsea_to_evidence] OK
  wrote: /work/paper/source_data/PANCAN_TP53_v1/evidence_tables/HNSC.evidence_table.tsv 
  n_terms: 50 


In [136]:
# Fig2 run
!python paper/scripts/run_fig2_pipeline.py \
  --cancers HNSC \
  --variants ours,shuffled_context \
  --gate-modes hard \
  --taus 0.2,0.4,0.6,0.8,0.9 \
  --force

[run_fig2] plan
  benchmark_id: PANCAN_TP53_v1
  cancers: HNSC
  variants: ours,shuffled_context
  gate_modes: hard
  taus: 0.20,0.40,0.60,0.80,0.90
  k_claims: 100
  seed: 42
  out_root: /work/paper/source_data/PANCAN_TP53_v1/out
[run_fig2] jobs: 10
[run_fig2] (1/10) HNSC ours gate=hard tau=0.20
[run_fig2] (2/10) HNSC shuffled_context gate=hard tau=0.20
[run_fig2] (3/10) HNSC ours gate=hard tau=0.40
[run_fig2] (4/10) HNSC shuffled_context gate=hard tau=0.40
[run_fig2] (5/10) HNSC ours gate=hard tau=0.60
[run_fig2] (6/10) HNSC shuffled_context gate=hard tau=0.60
[run_fig2] (7/10) HNSC ours gate=hard tau=0.80
[run_fig2] (8/10) HNSC shuffled_context gate=hard tau=0.80
[run_fig2] (9/10) HNSC ours gate=hard tau=0.90
[run_fig2] (10/10) HNSC shuffled_context gate=hard tau=0.90
[run_fig2] OK
  out_root: /work/paper/source_data/PANCAN_TP53_v1/out


In [137]:
!ls /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard

tau_0.20  tau_0.40  tau_0.60  tau_0.80	tau_0.90


In [138]:
!cat /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/run_meta.json
!cat /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/sample_card.tau.json


{
  "tool": "llm-pathway-curator",
  "cmd": "run",
  "run_id": "1769045002_557ae5e39f",
  "status": "ok",
  "started_epoch": 1769045002.6556568,
  "config": {
    "evidence_table": "/work/paper/source_data/PANCAN_TP53_v1/evidence_tables/HNSC.evidence_table.tsv",
    "sample_card": "/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/sample_card.tau.json",
    "outdir": "/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80",
    "force": true,
    "seed": 42,
    "run_meta_name": "run_meta.json",
    "tau": 0.8,
    "k_claims": 100
  },
  "env": {
    "python": "3.12.12 | packaged by conda-forge | (main, Oct 22 2025, 23:16:53) [GCC 14.3.0]",
    "platform": "Linux-6.12.54-linuxkit-aarch64-with-glibc2.39",
    "pandas": "2.3.3"
  },
  "inputs": {
    "evidence_table": {
      "path": "/work/paper/source_data/PANCAN_TP53_v1/evidence_tables/HNSC.evidence_table.tsv",
      "size_bytes": 12916,
      "mtime_epoch": 1769037125.8231242
    },
    "sample_

In [139]:
!python paper/scripts/fig2_sanity_check.py /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.20


BASE: /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.20
report   True 61911
audit    True 51746
modules  True 14057
edges    True 88967
terms    True 3188
distilled True 40208
risk     True 861
card     True 498
meta     True 4389

[run_meta] tool/status/step: llm-pathway-curator ok report_jsonl

[report] shape: (40, 45)
[report] decisions: {'PASS': 40}
[report] tau unique: [0.2]
[report] columns (top 25): ['created_at', 'run_id', 'method', 'disease', 'cancer', 'comparison', 'tau', 'species', 'schema_version', 'claim_id', 'decision_status', 'decision', 'survival', 'claim.entity_type', 'claim.entity_id', 'claim.context.disease', 'claim.context.tissue', 'claim.context.perturbation', 'claim.context.comparison', 'evidence_refs.gene_set_hash', 'benchmark_id', 'variant', 'gate_mode', 'claim.claim_id', 'claim.entity']

[report] contract view (head):
schema_version   benchmark_id                run_id method cancer          comparison  tau       claim_id decision  surviva

In [140]:
!python paper/scripts/fig2_sanity_check.py /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80

BASE: /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80
report   True 61915
audit    True 51744
modules  True 14057
edges    True 88967
terms    True 3188
distilled True 40208
risk     True 861
card     True 498
meta     True 4389

[run_meta] tool/status/step: llm-pathway-curator ok report_jsonl

[report] shape: (40, 45)
[report] decisions: {'PASS': 38, 'ABSTAIN': 2}
[report] tau unique: [0.8]
[report] columns (top 25): ['created_at', 'run_id', 'method', 'disease', 'cancer', 'comparison', 'tau', 'species', 'schema_version', 'claim_id', 'decision_status', 'decision', 'survival', 'claim.entity_type', 'claim.entity_id', 'claim.context.disease', 'claim.context.tissue', 'claim.context.perturbation', 'claim.context.comparison', 'evidence_refs.gene_set_hash', 'benchmark_id', 'variant', 'gate_mode', 'claim.claim_id', 'claim.entity']

[report] contract view (head):
schema_version   benchmark_id                run_id method cancer          comparison  tau       claim_id dec

In [92]:
for c in ("HNSC", "LUAD", "LUSC", "BRCA", "OV",  "UCEC",  "SKCM"):
    !Rscript paper/scripts/fig2_fgsea_to_evidence_table.R $c

[fgsea_to_evidence] inputs
  cancer: HNSC 
  rank: /work/paper/source_data/PANCAN_TP53_v1/derived/rankings/HNSC.deg_ranking.tsv 
  out : /work/paper/source_data/PANCAN_TP53_v1/evidence_tables/HNSC.evidence_table.tsv 
  msigdbr_version: 25.1.1 
  fgsea_version  : 1.36.2 
[fgsea_to_evidence] stats diagnostics
  stats_len: 20262 
  stats_type: double 
  stats_na: 0 
  stats_range: -12.6721379948078,10.5253182402821 
  gene_example: 4060,6817,5153,1902,19594,9055,6447,16500,1970,8301 
  gene_id_type: entrez
[fgsea_to_evidence] msigdbr entrez column: ncbi_gene 
[fgsea_to_evidence] overlap diagnostics
  overlap_summary: min 24 p25 80 median 141 p75 165 max 193 
  pathways_with_overlap>=15: 50 / 50 
[fgsea_to_evidence] fgsea diagnostics
  n_pathways: 50 
  res_rows  : 50 
[fgsea_to_evidence] OK
  wrote: /work/paper/source_data/PANCAN_TP53_v1/evidence_tables/HNSC.evidence_table.tsv 
  n_terms: 50 
[fgsea_to_evidence] inputs
  cancer: LUAD 
  rank: /work/paper/source_data/PANCAN_TP53_v1/derived

In [142]:
!python paper/scripts/run_fig2_pipeline.py \
  --cancers HNSC,LUAD,LUSC,BRCA,OV,UCEC,SKCM \
  --variants ours,shuffled_context \
  --gate-modes hard \
  --taus 0.2,0.4,0.6,0.8,0.9 \
  --force

[run_fig2] plan
  benchmark_id: PANCAN_TP53_v1
  cancers: HNSC,LUAD,LUSC,BRCA,OV,UCEC,SKCM
  variants: ours,shuffled_context
  gate_modes: hard
  taus: 0.20,0.40,0.60,0.80,0.90
  k_claims: 100
  seed: 42
  out_root: /work/paper/source_data/PANCAN_TP53_v1/out
[run_fig2] jobs: 70
[run_fig2] (1/70) HNSC ours gate=hard tau=0.20
[run_fig2] (2/70) HNSC shuffled_context gate=hard tau=0.20
[run_fig2] (3/70) HNSC ours gate=hard tau=0.40
[run_fig2] (4/70) HNSC shuffled_context gate=hard tau=0.40
[run_fig2] (5/70) HNSC ours gate=hard tau=0.60
[run_fig2] (6/70) HNSC shuffled_context gate=hard tau=0.60
[run_fig2] (7/70) HNSC ours gate=hard tau=0.80
[run_fig2] (8/70) HNSC shuffled_context gate=hard tau=0.80
[run_fig2] (9/70) HNSC ours gate=hard tau=0.90
[run_fig2] (10/70) HNSC shuffled_context gate=hard tau=0.90
[run_fig2] (11/70) LUAD ours gate=hard tau=0.20
[run_fig2] (12/70) LUAD shuffled_context gate=hard tau=0.20
[run_fig2] (13/70) LUAD ours gate=hard tau=0.40
[run_fig2] (14/70) LUAD shuffled_c

In [143]:
!python paper/scripts/fig2_make_labels_template.py \
  --root paper/source_data/PANCAN_TP53_v1/out \
  --out  paper/source_data/PANCAN_TP53_v1/labels.tsv \
  --per-run 20 \
  --prefer-status PASS \
  --with-hints

[OK] wrote: paper/source_data/PANCAN_TP53_v1/labels.tsv rows=150
[NEXT] Fill human_label with: ACCEPT / REJECT / SHOULD_ABSTAIN (partial labels ok)


In [141]:
!python paper/scripts/fig2_collect_risk_coverage.py \
  --root /work/paper/source_data/PANCAN_TP53_v1/out \
  --out  /work/paper/source_data/PANCAN_TP53_v1/out/fig2_risk_coverage.tsv \
  --benchmark-id PANCAN_TP53_v1


[OK] wrote: /work/paper/source_data/PANCAN_TP53_v1/out/fig2_risk_coverage.tsv rows=70
[WARN] n_pass_labeled is 0 for all rows. Use risk_proxy or provide --labels.


In [144]:
!python paper/scripts/bench_fig2.py \
  --reports paper/source_data/PANCAN_TP53_v1/out \
  --labels  paper/source_data/PANCAN_TP53_v1/labels.tsv \
  --out     /work/paper/source_data/PANCAN_TP53_v1/out/fig2_risk_coverage.tsv \
  --taus    0:1:0.05 \
  --rater-id R1


Traceback (most recent call last):
  File "/work/paper/scripts/bench_fig2.py", line 315, in <module>
    main()
  File "/work/paper/scripts/bench_fig2.py", line 303, in main
    reports_df = load_reports(report_paths)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/work/paper/scripts/bench_fig2.py", line 123, in load_reports
    raise ValueError(
ValueError: Missing metrics.survival in paper/source_data/PANCAN_TP53_v1/out/BRCA/ours/gate_hard/tau_0.20/report.jsonl line 1 (claim_id=c_a1ed09ef8099)


In [104]:
!python paper/scripts/fig2_plot_multipanel.py \
  --in  /work/paper/source_data/PANCAN_TP53_v1/out/fig2_risk_coverage.tsv \
  --out paper/source_data/PANCAN_TP53_v1/fig/Fig2_Supp_multipanel_proxy.pdf \
  --ycol risk_proxy \
  --benchmark-id PANCAN_TP53_v1 \
  --cols 4 \
  --fontsize 16 \
  --title "Risk–coverage curves across cancers"
